In [96]:
%run model_sucrose.ipynb

Knocking out reactions mentioned in the paper by Wu et al (Wu et al., 2019):
- LDH which converts pyruvate to lactate
- PTA which converts Acetyl-CoA to Acetyl phosphate

### Finding knockout targets

#### Finding the LDH reaction:

In [97]:
model.reactions.query("LDH")

[<Reaction BETALDHx at 0x1f606403910>,
 <Reaction LDH2 at 0x1f6066d9760>,
 <Reaction LDH_L at 0x1f606700e80>,
 <Reaction VALDHr at 0x1f606953df0>]

Checking if LDH_L is the one producing lactate

In [98]:
model.reactions.LDH_L.metabolites

{<Metabolite lac__L_c at 0x1f605eca850>: -1.0,
 <Metabolite nad_c at 0x1f605ee1a00>: -1.0,
 <Metabolite h_c at 0x1f605e9b9a0>: 1.0,
 <Metabolite nadh_c at 0x1f605ee1c40>: 1.0,
 <Metabolite pyr_c at 0x1f605f2a370>: 1.0}

In [99]:
model.metabolites.lac__L_c.name

'L-Lactate'

Yes. Seems like LDH_L is the reaction we're looking for.

#### Finding the PTA reaction

In [100]:
model.reactions.query("PTA")

[<Reaction APTA1i at 0x1f60630f9d0>,
 <Reaction ASPTA at 0x1f606403eb0>,
 <Reaction GF6PTA at 0x1f60658ae20>,
 <Reaction PTAr at 0x1f606793e80>,
 <Reaction PTA2 at 0x1f6067a9f70>]

Checking if PTAr is the reaction Acetyl phosphate:

In [101]:
model.reactions.PTAr.metabolites

{<Metabolite accoa_c at 0x1f605d79550>: -1.0,
 <Metabolite pi_c at 0x1f605ecacd0>: -1.0,
 <Metabolite actp_c at 0x1f605d99b50>: 1.0,
 <Metabolite coa_c at 0x1f605db4760>: 1.0}

In [102]:
model.metabolites.actp_c.name

'Acetyl phosphate'

Yes. Seems like PTAr is the reaction we're looking for.

### Knocking out reactions

Now we'll try knocking it out each of the reactions and see what happens to the growth and surfactin yield:

Knockout of LDH_L:

In [103]:
with model.copy():
    model.reactions.LDH_L.bounds = 0, 0
    print("LDH_L's bounds inside the with statement:")
    print(model.reactions.LDH_L.lower_bound, model.reactions.LDH_L.bounds)
    model.objective = model.reactions.BIOMASS_BS_10
    print('Mutant growth rate [mmol gDW^-1 h^-1]: ', model.optimize().objective_value)
    model.objective = model.reactions.SP_1
    surfactin_production = model.optimize().objective_value
    print("Theoretical max. yield [mmol-succ / mmol-ac]:", surfactin_production / (-1*model.reactions.EX_sucr_e.flux))

LDH_L's bounds inside the with statement:
0 (0, 0)
Mutant growth rate [mmol gDW^-1 h^-1]:  0.6241848146321101
Theoretical max. yield [mmol-succ / mmol-ac]: 0.11749294187350894


Knockout of PTAr:

In [104]:
model.copy()

Name,iYO844
Memory address,0x01f606bb5c10
Number of metabolites,991
Number of reactions,1255
Number of groups,0
Objective expression,1.0*SP_1 - 1.0*SP_1_reverse_84700
Compartments,"cytosol, extracellular space"


In [105]:
with model:
    model.reactions.PTAr.bounds = 0, 0
    print("LDH_L's bounds inside the with statement:")
    print(model.reactions.PTAr.lower_bound, model.reactions.PTAr.bounds)
    model.objective = model.reactions.BIOMASS_BS_10
    print('Mutant growth rate [mmol gDW^-1 h^-1]: ', model.optimize().objective_value)
    model.objective = model.reactions.SP_1
    surfactin_production = model.optimize().objective_value
    print("Theoretical max. yield [mmol-succ / mmol-ac]:", surfactin_production / (-1*model.reactions.EX_sucr_e.flux))

LDH_L's bounds inside the with statement:
0 (0, 0)
Mutant growth rate [mmol gDW^-1 h^-1]:  0.6241848146321086
Theoretical max. yield [mmol-succ / mmol-ac]: 0.11763020637816453


In both cases we see an increase in yield

In [106]:
with model:
    model.reactions.LDH_L.bounds = 0, 0
    model.reactions.PTAr.bounds = 0, 0
    print("LDH_L's bounds inside the with statement:")
    print(model.reactions.LDH_L.lower_bound, model.reactions.LDH_L.bounds)
    print(model.reactions.PTAr.lower_bound, model.reactions.PTAr.bounds)
    model.objective = model.reactions.BIOMASS_BS_10
    print('Mutant growth rate [mmol gDW^-1 h^-1]: ', model.optimize().objective_value)
    model.objective = model.reactions.SP_1
    surfactin_production = model.optimize().objective_value
    print("Theoretical max. yield [mmol-succ / mmol-ac]:", surfactin_production / (-1*model.reactions.EX_sucr_e.flux))

LDH_L's bounds inside the with statement:
0 (0, 0)
0 (0, 0)
Mutant growth rate [mmol gDW^-1 h^-1]:  0.624184814632109
Theoretical max. yield [mmol-succ / mmol-ac]: 0.11731806782561004
